# Modèle GRU-SVM pour Classification Binaire

Ce notebook implémente l'architecture GRU-SVM décrite dans l'article, combinant un réseau de neurones récurrent (GRU) avec une machine à vecteurs de support (SVM).

**Auteur:** maramchebbi  
**Date:** 2025-11-19  
**Plateforme:** Google Colab

## 1. Installation et Importation des Bibliothèques

In [ ]:
# Installation des packages nécessaires
!pip install -q tensorflow scikit-learn numpy pandas matplotlib seaborn

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.svm import LinearSVC
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, GRU, Dropout, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import warnings
warnings.filterwarnings('ignore')

# Configuration pour la reproductibilité
np.random.seed(42)
tf.random.set_seed(42)

print(f"TensorFlow version: {tf.__version__}")
print(f"GPU disponible: {tf.config.list_physical_devices('GPU')}")

## 2. Hyperparamètres du Modèle GRU-SVM

Les hyperparamètres suivants sont définis selon le **Tableau 1** de l'article.

In [ ]:
# Hyperparamètres selon le tableau 1
HYPERPARAMETERS = {
    'batch_size': 128,
    'cell_size': 128,          # Taille de la cellule GRU
    'dropout_rate': 0.5,
    'epochs': 3000,
    'learning_rate': 1e-3,
    'norm': 'L2',
    'svm_c': 5,                # Paramètre C du SVM
    'validation_split': 0.2,
    'early_stopping_patience': 50,
    'reduce_lr_patience': 20
}

# Affichage des hyperparamètres
print("="*60)
print("HYPERPARAMÈTRES DU MODÈLE GRU-SVM")
print("="*60)
for param, value in HYPERPARAMETERS.items():
    print(f"{param:.<30} {value}")
print("="*60)

## 3. Visualisation des Hyperparamètres

In [ ]:
# Visualisation des hyperparamètres
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
fig.suptitle('Hyperparamètres du Modèle GRU-SVM', fontsize=16, fontweight='bold')

# Batch Size
ax1 = axes[0, 0]
ax1.bar(['Batch Size'], [HYPERPARAMETERS['batch_size']], color='#2E86AB')
ax1.set_ylabel('Valeur')
ax1.set_title('Batch Size')
ax1.text(0, HYPERPARAMETERS['batch_size']/2, str(HYPERPARAMETERS['batch_size']), 
         ha='center', va='center', fontsize=14, color='white', fontweight='bold')

# Cell Size (GRU)
ax2 = axes[0, 1]
ax2.bar(['Cell Size'], [HYPERPARAMETERS['cell_size']], color='#A23B72')
ax2.set_ylabel('Valeur')
ax2.set_title('Taille de la Cellule GRU')
ax2.text(0, HYPERPARAMETERS['cell_size']/2, str(HYPERPARAMETERS['cell_size']), 
         ha='center', va='center', fontsize=14, color='white', fontweight='bold')

# Dropout Rate
ax3 = axes[0, 2]
ax3.bar(['Dropout Rate'], [HYPERPARAMETERS['dropout_rate']], color='#F18F01')
ax3.set_ylabel('Taux')
ax3.set_title('Taux de Dropout')
ax3.set_ylim([0, 1])
ax3.text(0, HYPERPARAMETERS['dropout_rate']/2, f"{HYPERPARAMETERS['dropout_rate']:.1f}", 
         ha='center', va='center', fontsize=14, color='white', fontweight='bold')

# Epochs
ax4 = axes[1, 0]
ax4.bar(['Epochs'], [HYPERPARAMETERS['epochs']], color='#C73E1D')
ax4.set_ylabel('Nombre')
ax4.set_title('Nombre d\'Epochs')
ax4.text(0, HYPERPARAMETERS['epochs']/2, str(HYPERPARAMETERS['epochs']), 
         ha='center', va='center', fontsize=14, color='white', fontweight='bold')

# Learning Rate
ax5 = axes[1, 1]
ax5.bar(['Learning Rate'], [HYPERPARAMETERS['learning_rate']], color='#6A994E')
ax5.set_ylabel('Valeur')
ax5.set_title('Taux d\'Apprentissage')
ax5.set_yscale('log')
ax5.text(0, HYPERPARAMETERS['learning_rate'], f"{HYPERPARAMETERS['learning_rate']}", 
         ha='center', va='bottom', fontsize=12, fontweight='bold')

# SVM C Parameter
ax6 = axes[1, 2]
ax6.bar(['SVM C'], [HYPERPARAMETERS['svm_c']], color='#BC4B51')
ax6.set_ylabel('Valeur')
ax6.set_title('Paramètre C du SVM (Régularisation)')
ax6.text(0, HYPERPARAMETERS['svm_c']/2, str(HYPERPARAMETERS['svm_c']), 
         ha='center', va='center', fontsize=14, color='white', fontweight='bold')

plt.tight_layout()
plt.show()

# Tableau récapitulatif
print("\n" + "="*60)
print("TABLEAU RÉCAPITULATIF DES HYPERPARAMÈTRES")
print("="*60)
df_params = pd.DataFrame({
    'Hyperparamètre': list(HYPERPARAMETERS.keys()),
    'Valeur': list(HYPERPARAMETERS.values())
})
print(df_params.to_string(index=False))
print("="*60)

## 4. Architecture du Modèle GRU-SVM

### Équations du GRU:

**Update Gate:**
$$z_t = \sigma(W_z \cdot [h_{t-1}, x_t])$$

**Reset Gate:**
$$r_t = \sigma(W_r \cdot [h_{t-1}, x_t])$$

**Candidate Hidden State:**
$$\tilde{h}_t = \tanh(W \cdot [r_t * h_{t-1}, x_t])$$

**New Hidden State:**
$$h_t = (1 - z_t) * h_{t-1} + z_t * \tilde{h}_t$$

### Fonction de Décision:
$$y' = \text{argmax}(\text{sign}(wx + b))$$

In [ ]:
class GRUSVM:
    """
    Architecture GRU-SVM pour classification binaire.
    
    Le modèle combine:
    - Une couche GRU pour l'extraction de caractéristiques temporelles
    - Un SVM linéaire (L2-SVM) pour la classification finale
    
    Équations du GRU:
    - z = σ(W_z · [h_{t-1}, x_t])           (update gate)
    - r = σ(W_r · [h_{t-1}, x_t])           (reset gate)
    - h̃_t = tanh(W · [r_t * h_{t-1}, x_t]) (candidate value)
    - h_t = (1 - z_t) * h_{t-1} + z_t * h̃_t (new hidden state)
    """
    
    def __init__(self, input_shape, hyperparams):
        self.input_shape = input_shape
        self.hyperparams = hyperparams
        self.gru_model = None
        self.svm_model = None
        self.scaler = StandardScaler()
        self.history = None
        
    def build_gru_feature_extractor(self):
        """
        Construit le réseau GRU pour l'extraction de caractéristiques.
        """
        inputs = Input(shape=self.input_shape, name='input_layer')
        
        # Couche GRU avec return_sequences=False pour obtenir le dernier état caché
        x = GRU(
            units=self.hyperparams['cell_size'],
            activation='tanh',
            recurrent_activation='sigmoid',
            use_bias=True,
            kernel_initializer='glorot_uniform',
            recurrent_initializer='orthogonal',
            return_sequences=False,
            name='gru_layer'
        )(inputs)
        
        # Dropout pour régularisation
        x = Dropout(self.hyperparams['dropout_rate'], name='dropout_layer')(x)
        
        # Le GRU model retourne les caractéristiques (h_t)
        self.gru_model = Model(inputs=inputs, outputs=x, name='GRU_Feature_Extractor')
        
        return self.gru_model
    
    def compile_gru(self):
        """
        Compile le modèle GRU avec l'optimiseur Adam.
        """
        optimizer = Adam(learning_rate=self.hyperparams['learning_rate'])
        
        # Pour l'entraînement du GRU seul (si nécessaire)
        self.gru_model.compile(
            optimizer=optimizer,
            loss='binary_crossentropy',
            metrics=['accuracy']
        )
    
    def train_gru(self, X_train, y_train, X_val=None, y_val=None):
        """
        Entraîne le modèle GRU.
        """
        callbacks = [
            EarlyStopping(
                monitor='val_loss',
                patience=self.hyperparams['early_stopping_patience'],
                restore_best_weights=True,
                verbose=1
            ),
            ReduceLROnPlateau(
                monitor='val_loss',
                factor=0.5,
                patience=self.hyperparams['reduce_lr_patience'],
                min_lr=1e-7,
                verbose=1
            )
        ]
        
        validation_data = (X_val, y_val) if X_val is not None else None
        
        self.history = self.gru_model.fit(
            X_train, y_train,
            batch_size=self.hyperparams['batch_size'],
            epochs=self.hyperparams['epochs'],
            validation_data=validation_data,
            callbacks=callbacks,
            verbose=1
        )
        
        return self.history
    
    def extract_features(self, X):
        """
        Extrait les caractéristiques en utilisant le GRU entraîné.
        Retourne h_t (le dernier état caché du GRU).
        """
        return self.gru_model.predict(X, batch_size=self.hyperparams['batch_size'])
    
    def train_svm(self, X_train, y_train):
        """
        Entraîne le SVM L2 sur les caractéristiques extraites par le GRU.
        
        Fonction de décision: f(x) = argmax(sign(wx + b))
        Avec w et b appris par le L2-SVM.
        """
        # Extraire les caractéristiques avec le GRU
        print("\nExtraction des caractéristiques d'entraînement avec GRU...")
        features_train = self.extract_features(X_train)
        
        # Normalisation des caractéristiques
        features_train_scaled = self.scaler.fit_transform(features_train)
        
        # Entraînement du SVM avec norme L2
        print("Entraînement du SVM L2...")
        self.svm_model = LinearSVC(
            C=self.hyperparams['svm_c'],
            penalty='l2',  # Régularisation L2
            loss='squared_hinge',  # Loss function du L2-SVM
            max_iter=5000,
            random_state=42,
            verbose=1
        )
        
        # Convertir les labels en -1 et +1 pour le SVM
        y_train_svm = np.where(y_train == 0, -1, 1)
        
        self.svm_model.fit(features_train_scaled, y_train_svm)
        
        print("Entraînement du SVM terminé!")
        
    def predict(self, X):
        """
        Prédit les classes en utilisant l'architecture GRU-SVM complète.
        
        y' = argmax(sign(wx + b))
        """
        # Extraire les caractéristiques
        features = self.extract_features(X)
        features_scaled = self.scaler.transform(features)
        
        # Prédire avec le SVM
        predictions = self.svm_model.predict(features_scaled)
        
        # Convertir -1, +1 en 0, 1
        predictions = np.where(predictions == -1, 0, 1)
        
        return predictions
    
    def predict_proba(self, X):
        """
        Retourne les scores de décision du SVM.
        """
        features = self.extract_features(X)
        features_scaled = self.scaler.transform(features)
        
        # Decision function donne les scores
        scores = self.svm_model.decision_function(features_scaled)
        
        return scores
    
    def summary(self):
        """
        Affiche le résumé de l'architecture.
        """
        print("\n" + "="*60)
        print("ARCHITECTURE GRU-SVM")
        print("="*60)
        self.gru_model.summary()
        print("\n" + "="*60)
        print("SVM CLASSIFIER")
        print("="*60)
        print(f"Kernel: Linear (L2-SVM)")
        print(f"C parameter: {self.hyperparams['svm_c']}")
        print(f"Penalty: L2")
        print("="*60)

## 5. Génération de Données Synthétiques (Exemple)

⚠️ **Note importante:** Pour votre cas réel, remplacez cette section par le chargement de vos données de diagnostic du cancer.

In [ ]:
def generate_synthetic_data(n_samples=38400, n_timesteps=50, n_features=10):
    """
    Génère des données synthétiques pour tester le modèle.
    
    Pour votre cas réel, remplacez cette fonction par le chargement
    de vos données de diagnostic du cancer.
    """
    print(f"\nGénération de {n_samples} échantillons synthétiques...")
    print(f"Dimensions: ({n_timesteps} timesteps, {n_features} features)")
    
    # Génération de séquences temporelles
    X = np.random.randn(n_samples, n_timesteps, n_features)
    
    # Génération de labels binaires (0: benign, 1: malignant)
    # Ajouter une certaine structure pour que le modèle puisse apprendre
    y = (np.mean(X[:, :, 0], axis=1) + np.std(X[:, :, 1], axis=1) > 0).astype(int)
    
    print(f"Distribution des classes:")
    print(f"  Classe 0 (Benign): {np.sum(y == 0)} ({np.mean(y == 0)*100:.2f}%)")
    print(f"  Classe 1 (Malignant): {np.sum(y == 1)} ({np.mean(y == 1)*100:.2f}%)")
    
    return X, y

# Génération des données
X, y = generate_synthetic_data(n_samples=38400, n_timesteps=50, n_features=10)

# Division des données
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp
)

print(f"\nTaille des ensembles:")
print(f"  Train: {X_train.shape[0]} samples")
print(f"  Validation: {X_val.shape[0]} samples")
print(f"  Test: {X_test.shape[0]} samples")

## 6. Construction et Entraînement du Modèle

In [ ]:
# Instanciation du modèle
input_shape = (X_train.shape[1], X_train.shape[2])  # (timesteps, features)

gru_svm_model = GRUSVM(input_shape=input_shape, hyperparams=HYPERPARAMETERS)

# Construction de l'extracteur de caractéristiques GRU
gru_svm_model.build_gru_feature_extractor()
gru_svm_model.compile_gru()

# Affichage de l'architecture
gru_svm_model.summary()

In [ ]:
# Entraînement du GRU
print("\n" + "="*60)
print("PHASE 1: ENTRAÎNEMENT DU GRU")
print("="*60)

history = gru_svm_model.train_gru(
    X_train, y_train,
    X_val, y_val
)

## 7. Visualisation de l'Entraînement du GRU

In [ ]:
# Visualisation de la courbe d'apprentissage
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Loss
axes[0].plot(history.history['loss'], label='Train Loss', linewidth=2)
axes[0].plot(history.history['val_loss'], label='Validation Loss', linewidth=2)
axes[0].set_xlabel('Epoch', fontsize=12)
axes[0].set_ylabel('Loss', fontsize=12)
axes[0].set_title('Courbe de Loss du GRU', fontsize=14, fontweight='bold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Accuracy
axes[1].plot(history.history['accuracy'], label='Train Accuracy', linewidth=2)
axes[1].plot(history.history['val_accuracy'], label='Validation Accuracy', linewidth=2)
axes[1].set_xlabel('Epoch', fontsize=12)
axes[1].set_ylabel('Accuracy', fontsize=12)
axes[1].set_title('Courbe d\'Accuracy du GRU', fontsize=14, fontweight='bold')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Entraînement du SVM sur les caractéristiques GRU
print("\n" + "="*60)
print("PHASE 2: ENTRAÎNEMENT DU SVM L2")
print("="*60)

gru_svm_model.train_svm(X_train, y_train)

## 8. Évaluation du Modèle

In [ ]:
# Prédictions sur l'ensemble de test
print("\n" + "="*60)
print("ÉVALUATION SUR L'ENSEMBLE DE TEST")
print("="*60)

y_pred = gru_svm_model.predict(X_test)

# Calcul des métriques
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)

# Calcul des métriques détaillées
tn, fp, fn, tp = conf_matrix.ravel()
tpr = tp / (tp + fn)  # True Positive Rate (Sensitivity)
tnr = tn / (tn + fp)  # True Negative Rate (Specificity)
fpr = fp / (fp + tn)  # False Positive Rate
fnr = fn / (fn + tp)  # False Negative Rate

print(f"\nAccuracy: {accuracy*100:.2f}%")
print(f"\nMétriques détaillées:")
print(f"  TPR (True Positive Rate):  {tpr*100:.2f}%")
print(f"  TNR (True Negative Rate):  {tnr*100:.2f}%")
print(f"  FPR (False Positive Rate): {fpr*100:.2f}%")
print(f"  FNR (False Negative Rate): {fnr*100:.2f}%")

print("\n" + classification_report(y_test, y_pred, 
                                    target_names=['Benign', 'Malignant']))

## 9. Visualisation de la Matrice de Confusion

In [ ]:
# Visualisation de la matrice de confusion
fig, ax = plt.subplots(1, 1, figsize=(8, 6))

sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['Benign', 'Malignant'],
            yticklabels=['Benign', 'Malignant'],
            cbar_kws={'label': 'Count'},
            ax=ax)

ax.set_xlabel('Classe Prédite', fontsize=12)
ax.set_ylabel('Classe Réelle', fontsize=12)
ax.set_title('Matrice de Confusion - Modèle GRU-SVM', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

## 10. Comparaison des Résultats avec le Tableau 2

In [ ]:
# Résultats attendus selon le tableau 2
expected_results = {
    'Model': 'GRU-SVM',
    'Accuracy': 93.75,
    'Data points': 384000,
    'Epochs': 3000,
    'FPR': 16.67,
    'FNR': 0,
    'TPR': 100,
    'TNR': 83.33
}

# Résultats obtenus
obtained_results = {
    'Model': 'GRU-SVM (Implementation)',
    'Accuracy': accuracy * 100,
    'Data points': X_train.shape[0],
    'Epochs': len(history.history['loss']),
    'FPR': fpr * 100,
    'FNR': fnr * 100,
    'TPR': tpr * 100,
    'TNR': tnr * 100
}

# Tableau de comparaison
comparison_df = pd.DataFrame([expected_results, obtained_results])

print("\n" + "="*60)
print("COMPARAISON DES RÉSULTATS")
print("="*60)
print(comparison_df.to_string(index=False))
print("="*60)

In [ ]:
# Visualisation comparative
metrics = ['Accuracy', 'TPR', 'TNR', 'FPR', 'FNR']
expected_values = [expected_results[m] for m in metrics]
obtained_values = [obtained_results[m] for m in metrics]

x = np.arange(len(metrics))
width = 0.35

fig, ax = plt.subplots(figsize=(12, 6))
bars1 = ax.bar(x - width/2, expected_values, width, label='Attendu (Article)', color='#2E86AB')
bars2 = ax.bar(x + width/2, obtained_values, width, label='Obtenu (Implementation)', color='#F18F01')

ax.set_ylabel('Pourcentage (%)', fontsize=12)
ax.set_title('Comparaison des Métriques: GRU-SVM', fontsize=14, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels(metrics)
ax.legend()
ax.grid(True, alpha=0.3, axis='y')

# Ajouter les valeurs sur les barres
for bars in [bars1, bars2]:
    for bar in bars:
        height = bar.get_height()
        ax.annotate(f'{height:.1f}%',
                    xy=(bar.get_x() + bar.get_width() / 2, height),
                    xytext=(0, 3),
                    textcoords="offset points",
                    ha='center', va='bottom', fontsize=9)

plt.tight_layout()
plt.show()

## 11. Sauvegarde du Modèle

In [ ]:
# Sauvegarder le modèle GRU
print("\nSauvegarde des modèles...")
gru_svm_model.gru_model.save('gru_feature_extractor.h5')
print("✓ Modèle GRU sauvegardé: gru_feature_extractor.h5")

# Sauvegarder le SVM avec pickle
import pickle
with open('svm_classifier.pkl', 'wb') as f:
    pickle.dump(gru_svm_model.svm_model, f)
print("✓ Modèle SVM sauvegardé: svm_classifier.pkl")

# Sauvegarder le scaler
with open('feature_scaler.pkl', 'wb') as f:
    pickle.dump(gru_svm_model.scaler, f)
print("✓ Scaler sauvegardé: feature_scaler.pkl")

print("\nTous les modèles ont été sauvegardés avec succès!")

## 12. Téléchargement des Modèles (Google Colab)

In [ ]:
# Télécharger les fichiers sauvegardés dans Google Colab
from google.colab import files

print("Téléchargement des modèles...")
files.download('gru_feature_extractor.h5')
files.download('svm_classifier.pkl')
files.download('feature_scaler.pkl')
print("Téléchargement terminé!")

## 13. Chargement et Utilisation du Modèle

In [ ]:
# Exemple de chargement du modèle
def load_gru_svm_model():
    """
    Charge le modèle GRU-SVM sauvegardé.
    """
    from tensorflow.keras.models import load_model
    import pickle
    
    # Charger le GRU
    gru_model = load_model('gru_feature_extractor.h5')
    
    # Charger le SVM
    with open('svm_classifier.pkl', 'rb') as f:
        svm_model = pickle.load(f)
    
    # Charger le scaler
    with open('feature_scaler.pkl', 'rb') as f:
        scaler = pickle.load(f)
    
    return gru_model, svm_model, scaler

# Exemple d'utilisation
# gru_model, svm_model, scaler = load_gru_svm_model()
print("Fonction de chargement du modèle définie!")

## 14. Instructions pour Utiliser vos Propres Données

Pour utiliser ce notebook avec vos données de diagnostic du cancer:

### 1. Préparez vos données

```python
# Upload votre fichier de données sur Google Colab
from google.colab import files
uploaded = files.upload()

# Chargez vos données (exemple avec CSV)
import pandas as pd
df = pd.read_csv('votre_fichier.csv')

# Préparez X et y
# X doit avoir la forme: (n_samples, n_timesteps, n_features)
# y doit avoir la forme: (n_samples,) avec des valeurs 0 ou 1
```

### 2. Format des données

- **X (Features)**: Array 3D de forme `(n_samples, n_timesteps, n_features)`
  - `n_samples`: Nombre total d'échantillons
  - `n_timesteps`: Nombre de pas de temps (séquence)
  - `n_features`: Nombre de caractéristiques par pas de temps

- **y (Labels)**: Array 1D de forme `(n_samples,)`
  - `0`: Bénin (Benign)
  - `1`: Malin (Malignant)

### 3. Remplacez la section "Génération de Données Synthétiques"

Supprimez ou commentez la cellule de génération de données synthétiques et utilisez vos propres données.

## 15. Conclusion

Ce notebook implémente l'architecture **GRU-SVM** pour la classification binaire du cancer.

### Points clés:

1. ✅ **Architecture hybride**: GRU (extraction) + SVM (classification)
2. ✅ **Hyperparamètres optimisés**: Selon le Tableau 1
3. ✅ **Régularisation**: Dropout (0.5) et norme L2
4. ✅ **Visualisations complètes**: Hyperparamètres, courbes d'apprentissage, métriques
5. ✅ **Sauvegarde/Chargement**: Modèles réutilisables

### Équations implémentées:

**GRU:**
- $z_t = \sigma(W_z \cdot [h_{t-1}, x_t])$ (update gate)
- $r_t = \sigma(W_r \cdot [h_{t-1}, x_t])$ (reset gate)
- $\tilde{h}_t = \tanh(W \cdot [r_t * h_{t-1}, x_t])$ (candidate)
- $h_t = (1 - z_t) * h_{t-1} + z_t * \tilde{h}_t$ (new state)

**SVM:**
- $y' = \text{argmax}(\text{sign}(wx + b))$ (decision function)

### Performance attendue (selon Tableau 2):

- **Accuracy**: 93.75%
- **TPR**: 100%
- **TNR**: 83.33%
- **FPR**: 16.67%
- **FNR**: 0%

---

**Créé par:** maramchebbi  
**Date:** 2025-11-19  
**Plateforme:** Google Colab

📊 **Notebook prêt pour l'entraînement sur GPU!**